# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data. The sample orders file is Sample - Superstore dataset from Tableau Software.

If you have suggestions or improvements please contribute on https://github.com/joaolcorreia/RFM-analysis

In [1]:
import pandas as pd

Read the sample orders file, containing all past purchases for all customers.

In [5]:
orders = pd.read_csv('/Users/mitttttechka/Downloads/sample-orders.csv', encoding = "ISO-8859-1")

In [6]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franzsisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


## Create the RFM Table

Since recency is calculated for a point in time and the Tableau Super Store dataset last order date is Dec 31 2014, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [7]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [8]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

Create the RFM Table

In [9]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [10]:
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [11]:
aaron = orders[orders['customer']=='Aaron Bergman']
aaron

,order_date,order_id,customer,grand_total
624,2011-02-19,CA-2011-152905,Aaron Bergman,13
665,2011-03-07,CA-2011-156587,Aaron Bergman,310
2336,2013-11-11,CA-2013-140935,Aaron Bergman,564


## Determining RFM Quartiles

In [12]:
quantiles = rfmTable.quantile(q=[0.2, 0.4, 0.6, 0.8])

In [13]:
quantiles

,recency,frequency,monetary_value
0.2,26.0,4.0,961.4
0.4,53.0,5.0,1739.8
0.6,103.0,7.0,2750.8
0.8,224.6,8.6,4282.6


Send quantiles to a dictionary, easier to use.

In [14]:
quantiles = quantiles.to_dict()

In [15]:
quantiles

{'recency': {0.2: 26.0, 0.4: 53.0, 0.6: 103.0, 0.8: 224.60000000000002},
 'frequency': {0.2: 4.0, 0.4: 5.0, 0.6: 7.0, 0.8: 8.600000000000023},
 'monetary_value': {0.2: 961.4, 0.4: 1739.8, 0.6: 2750.8, 0.8: 4282.6}}

## Creating the RFM segmentation table

In [16]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [17]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 4
    else:
        return 5
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 2
    else:
        return 1


In [18]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [19]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [25]:
rfmSegmentation

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,5,5,5,555
Aaron Hawkins,12,7,1744,1,3,3,133
Aaron Smayling,88,7,3050,3,3,2,332
Adam Bellavance,54,8,7756,3,2,1,321
Adam Hart,34,10,3249,2,1,2,212
...,...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,3,213
Yana Sorensen,9,8,6721,1,2,1,121
Yoseph Carroll,4,5,5455,1,4,1,141


In [26]:
rfmSegmentation.columns

Index(['recency', 'frequency', 'monetary_value', 'R_Quartile', 'F_Quartile',
       'M_Quartile', 'RFMClass'],
      dtype='object')

In [35]:
rfmSegmentation['RFM_mean'] = round(rfmSegmentation[['R_Quartile', 'F_Quartile', 'M_Quartile']].mean(axis = 1), 2)

In [36]:
rfmSegmentation

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,RFM_mean
customer,,,,,,,,
Aaron Bergman,415,3,887,5,5,5,555,5.00
Aaron Hawkins,12,7,1744,1,3,3,133,2.33
Aaron Smayling,88,7,3050,3,3,2,332,2.67
Adam Bellavance,54,8,7756,3,2,1,321,2.00
Adam Hart,34,10,3249,2,1,2,212,1.67
...,...,...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,3,213,2.00
Yana Sorensen,9,8,6721,1,2,1,121,1.33
Yoseph Carroll,4,5,5455,1,4,1,141,2.00


In [38]:
rfmSegmentation = rfmSegmentation.drop(['R_Quartile', 'F_Quartile', 'M_Quartile'], axis = 1)

In [39]:
rfmSegmentation

,recency,frequency,monetary_value,RFMClass,RFM_mean
customer,,,,,
Aaron Bergman,415,3,887,555,5.00
Aaron Hawkins,12,7,1744,133,2.33
Aaron Smayling,88,7,3050,332,2.67
Adam Bellavance,54,8,7756,321,2.00
Adam Hart,34,10,3249,212,1.67
...,...,...,...,...,...
Xylona Preis,43,11,2375,213,2.00
Yana Sorensen,9,8,6721,121,1.33
Yoseph Carroll,4,5,5455,141,2.00


In [41]:
rfmSegmentation.to_csv('rfm-table.csv', sep=',')